### Exercise 7.6 Quasi-hyperbolic discounting and self-control

This note augments the environment studied in exercise 7.6 on Self-control of Ljungqvist and Sargent Recursive Macro textbook to an environment with productivity shocks. It is a partial equilibrium model of a present-biased household. The equilibrium concept is Markov Perfect Equilibrium. First, we revise the equilibrium concept.

##### Markov Perfect Equilibrium

Consider an economy with:
- 2 agents with utility $u_i(a_i, a_j, x)$ where $a_i$ denotes the action of $i$ and $x$ is the state variable.
- the future is discounted at rate $\beta$ so each agent, taking the strategy profile $a_{j}$ as given, solves:
\begin{align*}
\max_{(a_{it})_{t=0}^\infty} &\ \sum_{t=0}^\infty \beta^t \ u_i(a_i, a_j(h_t), x_t)\\
 & h_{t+1} = (h_t, (a_{it}, a_{jt}(h_t)))\\
 & x_{t+1} = f(a_{it}, a_{jt}(h_t), x_t)
\end{align*}
Note that each player takes into account the effect that her action has on the history and hence on the action of the other player but a player takes the strategy (not the sequence of actions) of the other player as given and unaffected by her choice of srategies.
- an action profile is Markovian if for every $i$, $a_i(x_t, h_{t-1}) = a_i(x_t, \tilde{h}_{t-1})$ for all histories $\tilde{h}_{t-1}$.
- the Markovian best response of an agent can be formulated in recursive form:
\begin{align*}
v_i(x) = \max_{a_{i}} & \ u_i(a_i, a_j(x), x) + \beta v_i(x')\\
 & x' = f(a_i, a_j(x), x)
\end{align*}

A Markov Perfect Equilibrium is a pair of value functions $(v_1, v_2)$ and Markovian best responses $(a_1, a_2)_{i=1,2}$ such that $v_i$ solves the functional equation for agent $i$ and $a_i$ is the associated policy function.

The above problems are difficult to solve because they involve interrelated value functions iterations.
The FOC are, for all $x$:
\begin{equation*}
\frac{\partial u_i(a_i, a_j(x), x)}{\partial a_i} + \beta v_i'(f(a_1, a_2, x)) \frac{\partial f(a_i, a_j(x), x)}{\partial a_i} = 0
\end{equation*}
and by Benveniste-Scheinckman,
\begin{equation*}
v_i'(x') = \frac{\partial u_i(a_i, a_j(x'), x')}{\partial a_j} \frac{\partial a_j}{\partial x} + \frac{\partial u_i(a_i, a_j(x'), x')}{\partial x} 
\end{equation*}

If the problem is linear quadratic, then the interrelated Belleman equation becomes stacked Riccati equations which is easy to solve with the method of undetermined coefficients. This is particularly useful for oligopoli models for instance. Below we solve a single planner problem who has time-inconsistent preferences due to a present-bias modelled with quasi-hyperbolic (also known as $\beta \delta$) preferences. The time-inconsistency in preferences makes it a game played between different selves of the same planner.

### Present-biased household

- finite horizon $T$
- representative household (with $T+1$ selves)
- inelastic labor normalized to $1$
- one sector model with full depreciation
\begin{equation}
c_t + k_{t+1} = \xi_t f(k_t)
\end{equation}
with $f' > 0$ and $f'' \leq 0$ and $\xi_t \sim_{iid} \phi$.

Preferences are:
- at $T$: 
\begin{equation}
u(c_T)
\end{equation}
- at $T-1$: 
\begin{equation}
u(c_{T-1}) + \beta \delta E[u(c_T)]
\end{equation}
- at $T-2$: 
\begin{equation}
u(c_{T-2}) + \beta E[\delta u(c_{T-1}) + \delta^2 u(c_T)]
\end{equation}
- etc

Note that the household has time-inconsistent preferences (i.e. not recursive): the time $t-$self discounts the next period with factor $\beta \delta$ and any two subsequent periods after that with factor $\delta$.

The household chooses $c_t$ at time $t$ and cannot commit to future choices. A household that can commit to future choices is said to have ``self-control''.

##### Remarks:
- finite horizon limits the set of possible equilibria. With infinite horizon, folks theorems suggest that we could support more equilibria (including self-control) with trigger strategies
- $T+1$ personalities makes us model the situation as a game played by different selves of the household

The solution is obtained by backward induction. 

#### Sequential formulation

\begin{align}
u(f(k_T)) \tag{period $T$}\\
\max_k u(f(k_{T-1}) - k) + \beta \delta E[u(f(k))] \tag{period $T-1$}\\
\max_k u(f(k_{T-2}) - k) + \beta E[\delta u(f(k_{T-1}(k))) + \delta^2 u(f(k_{T}(k_{T-1}(k))))] \tag{period $T-1$}
\end{align}

The sequential formulation makes it clear that the current self anticipates the effect of its choice on the choice of future selves, taking the strategy of future selves as given.

#### Recursive formulation

Different selves disagree on discounting. Each period, the value function $v_{T-t}$ records the value function of the period $T-t$-self and the value function $W_{T-t}$ record the continuation value for all other selves. With a finite horizon, we have a sequence of value functions.

Period $T$ is the last period:
\begin{align}
v_T(y) &= u(y)\\
W_T(y) &= u(y)
\end{align}

The index $t$ 

\begin{align}
v_{T-t}(y) &:= \max_{k'} u(y - k') + \beta  \delta E[W_{T-t+1}(\xi f(k'))]  \\
k_{T-t}'(k)&:= \arg\max_{k'} u(y - k') + \beta  \delta E[W_{T-t+1}(\xi f(k'))]  \\
W_{T-t}(y) &:= u(y - k_{T-t}'(k)) + \delta E[W_{T-t+1}(\xi f(k_{T-t}'(k)))]
\end{align}
where $k_{T-t}'(k)$ denotes the policy function from the right hand side the equation defining $v_{T-t}$.

In [1]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt
from interpolation import interp
from numba import jit, njit, jitclass, prange, float64, int32
from quantecon.optimize.scalar_maximization import brent_max
from scipy.optimize import brentq

%matplotlib inline

In [6]:
'This is to be able to use JIT from Numba to speed up the code, which will be done with at jitclass before the class'
qhyp_cons = [
    ('alpha', float64), # power of production function
    ('beta', float64), # inverse measure of present bias
    ('delta', float64), # discount factor            
    ('mu', float64),          # Shock location parameter
    ('s', float64),          # Shock scale parameter
    ('grid', float64[:]),    # Grid (array)
    ('shocks', float64[:]),  # Shock draws (array)
    ('T', int32)
]

In [7]:
@jitclass(qhyp_cons) #jitclass because we are building a class
class LS_qhyp_cons():
    def __init__(self,
                alpha = 0.4,
                beta= 0.96,
                delta = 0.96,
                mu = 0,
                s = 0.1,
                grid_max = 4,
                grid_size = 200,
                shock_size = 250,
                T = 10):
        self.alpha, self.beta, self.delta, self.mu, self.s, self.T = alpha, beta, delta, mu, s, T
        self.grid = np.linspace(1e-5, grid_max, grid_size) # set up the grid for the state y
        self.shocks = np.exp(mu + s * np.random.randn(shock_size)) # draw some shocks to take expectations
    
    def f(self, k):
        return k**self.alpha
    
    def f_prime(self, k):
        return self.alpha * k**(self.alpha - 1)
    
    def u(self, c):
        return np.log(c)
    
    def u_prime(self, c):
        return 1/c
    
    def RHS_Bellman(self, c, y, w_array):
        'returns the right hand side for a given choice c and a given state y'
        u, f, beta, delta, shocks = self.u, self.f, self.beta, self.delta, self.shocks
        w = lambda x: interp(self.grid, w_array, x)
        return u(c) + beta * delta * np.mean(w(f(y-c) * shocks)) #np.mean computes the expectation over shock_size draws
    
    def Update_cont_val(self, c, y, w_array):
        u, f, beta, delta, shocks = self.u, self.f, self.beta, self.delta, self.shocks
        w = lambda x: interp(self.grid, w_array, x)
        return u(c) + delta * np.mean(w(f(y-c) * shocks)) #np.mean computes the expectation over shock_size draws

The operator $v\_k\_w(W)$ computes, for each output $y$ on a grid, the current policy function $k'(\cdot)$, the associated value function $v$ for period $t$-self and the continuation value $w$ from the perspective of all other selves:
\begin{equation}
\left[
\begin{array}{cc}
v \\
k \\
W
\end{array}
\right]
= 
\left[
\begin{array}{cc}
u(y - k'(k)) + \beta  \delta E[W(\xi f(k'(k)))] \\
k'(k) \ \text{policy function v-greedy} \\
u(y - k'(k)) + \delta E[W(\xi f(k'(k)))]
\end{array}
\right]
\end{equation}

This operator computes . It also computes the continuation value function with geometric discounting.

In [8]:
@jit #(nopython=True) # jit because we are defining a function
def v_k_w(qh, W):
    ''' 
    qh is an instance of the class LS_qhyp_cons
    W is the value function discounted by delta, an array of size grid_size in the class LS_qhyp_cons
    '''
    v_previous = np.empty_like(W) # creates a vector of the same size as W that will record v_previous
    w_previous = np.empty_like(W)
    c_greedy = np.empty_like(W)
    v_k_w_sol = np.empty((3, len(W)))
    for i in range(len(qh.grid)):
        y = qh.grid[i]
        c_val, vprevious, info = brent_max(qh.RHS_Bellman, 1e-10, y, args=(y, W)) # maximize over c between 0 and y-1e-10
        v_previous[i] = vprevious
        c_greedy[i] = c_val
        w_previous[i] = qh.Update_cont_val(c_val, y, W)
        v_k_w_sol[0,:] = v_previous
        v_k_w_sol[1,:] = c_greedy
        v_k_w_sol[2,:] = w_previous
    return v_k_w_sol

The next cell creates matrices within which to store the equilibrium objects $(v_t, k'_t, W_t)_{t=0}^T$

In [15]:
beta_val = 1

# set up an instance
qh = LS_qhyp_cons(alpha = 0.4,
                beta= beta_val,
                delta = 0.96,
                mu = 0,
                s = 0.1,
                grid_max = 4,
                grid_size = 100,
                shock_size = 250,
                T = 4)

# Value function at time T which initiates the backward induction (it's the utility of all possible output values)
W_T = qh.u(qh.grid)         

# Record the T continuation value functions discounting with factor delta
W = np.empty((int(qh.T), len(W_T)))
W[qh.T-1,:] = W_T                # last value function is just the utility of all possible output values

# Record the T policy functions of self t at t
k_prime = np.empty((qh.T, len(W_T)))
k_prime[qh.T-1,:] = np.zeros(len(W_T))

# Record the T value functions of self t at t
V = np.empty((qh.T, len(W_T)))
V[qh.T-1,:] = W_T                # last value function is just the utility of all possible output values

The next cell computes an equilibrium and fills in the matrices from the previous cell

In [ ]:
num_iter = qh.T
for i in range(num_iter):
    vkw = v_k_w(qh, W[T-i,:])
    k_prime[qh.T-1,:] = 
    W[qh.T-1,:] = vkw[3,:]
    

### Test code with analytical solution when there is no hyperbolic discounting (i.e. $\beta = 1$)

##### Exact solution if there is no hyborlic discounting (i.e. $\beta=1$)

In [ ]:
def Exact_policy(qh, y):
    """
    True optimal policy
    """
    return (1 - qh.alpha * qh.delta) * y

def Exact_v(qh, y):
    """
    True value function
    """
    c1 = np.log(1 - qh.alpha * qh.delta) / (1 - qh.delta)
    c2 = (qh.mu + qh.alpha * np.log(qh.alpha * qh.delta)) / (1 - qh.alpha)
    c3 = 1 / (1 - qh.delta)
    c4 = 1 / (1 - qh.alpha * qh.delta)
    return c1 + c2 * (c3 - c4) + c4 * np.log(y)

When there is no hyperbolic discounting (i.e. $\beta = 1$), solving for the sequence of value and policy functions by backward induction is the same algorithm as Value Function Iteration for an infinite horizon. For an infinite horizon, we are interested in the fixed point of the Bellman equation. The sequence of value functions along the iteration to convergence is not relevant, only the fixed point matters. For a finite horizon, the sequence of value functions along the iteration is the object of interest; each iteration solves for the previous value function and the previous policy function.

In [ ]:
qh = LS_qhyp_cons(alpha = 0.4,
                beta= 1,
                delta = 0.96,
                mu = 0,
                s = 0.1,
                grid_max = 4,
                grid_size = 100,
                shock_size = 250)

fig, ax = plt.subplots(figsize=(10,8))
xgrid = qh.grid
ax.plot(xgrid, Exact_v(qh, xgrid), label='v')
vkw = v_k_w(qh, Exact_v(qh, xgrid))
WW = vkw[0,:]
ax.plot(xgrid, WW, label='v_k_w(v)')
W =  np.log(qh.grid)
num_iter = 50
for i in range(num_iter):
    vkw = v_k_w(qh, W)
    W = vkw[2,:]
    ax.plot(xgrid, W, color=plt.cm.jet(i / num_iter), lw=2, alpha=0.6)
ax.set_title('Exact solution is a fixed point of Bellman operator (no hyperbolic discounting)') 
ax.set_xlabel('y', fontsize=16)
ax.set_ylabel('v', fontsize=16)
ax.legend(loc='upper left')
plt.show()

### 